SPLIT THEM TO 3 FOLDERS

In [1]:
from pathlib import Path
import shutil
import json
import csv
import random
import re
from datetime import datetime
from collections import defaultdict

# =========================
# CONFIG
# =========================
INTERIM_DIR = Path("../../data/interim/Stage1/color_clahe_1500x1000_noborder_aug")  # source (contains *_orig.png and *_augXX.png)
PROCESSED_DIR = Path("../../data/processed/Stage1")

IMAGES_DIR = PROCESSED_DIR / "images"
TRAIN_DIR = IMAGES_DIR / "train"
VAL_DIR   = IMAGES_DIR / "val"
TEST_DIR  = IMAGES_DIR / "test"

TRAIN_RATIO = 0.85
VAL_RATIO   = 0.10
TEST_RATIO  = 0.05
SEED = 42

IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tiff"}

# =========================
# SETUP
# =========================
random.seed(SEED)

TRAIN_DIR.mkdir(parents=True, exist_ok=True)
VAL_DIR.mkdir(parents=True, exist_ok=True)
TEST_DIR.mkdir(parents=True, exist_ok=True)

image_paths = sorted([p for p in INTERIM_DIR.iterdir() if p.suffix.lower() in IMAGE_EXTS])
assert len(image_paths) > 0, "No images found in interim directory"
print(f"Found {len(image_paths)} images")

# =========================
# GROUPING (prevents leakage)
# =========================
# Group all siblings together:
#   base_orig.png, base_aug00.png, base_aug01.png ... -> same group "base"
def group_id_from_filename(filename: str) -> str:
    stem = Path(filename).stem
    # remove trailing _orig or _augXX
    stem = re.sub(r"_(orig|aug\d{2})$", "", stem)
    return stem

groups = defaultdict(list)
for p in image_paths:
    gid = group_id_from_filename(p.name)
    groups[gid].append(p)

group_ids = list(groups.keys())
random.shuffle(group_ids)

n_groups = len(group_ids)
n_train_g = int(n_groups * TRAIN_RATIO)
n_val_g   = int(n_groups * VAL_RATIO)

train_gids = set(group_ids[:n_train_g])
val_gids   = set(group_ids[n_train_g:n_train_g + n_val_g])
test_gids  = set(group_ids[n_train_g + n_val_g:])

train_imgs = [p for gid in train_gids for p in groups[gid]]
val_imgs   = [p for gid in val_gids   for p in groups[gid]]
test_imgs  = [p for gid in test_gids  for p in groups[gid]]

print(f"Groups total: {n_groups}")
print(f"Group split -> train/val/test: {len(train_gids)}/{len(val_gids)}/{len(test_gids)}")
print(f"Image split -> train/val/test: {len(train_imgs)}/{len(val_imgs)}/{len(test_imgs)}")

# =========================
# COPY IMAGES
# =========================
def copy_split(imgs, split_dir, split_name):
    relpaths = []
    for p in imgs:
        dst = split_dir / p.name
        shutil.copy2(p, dst)
        relpaths.append(f"images/{split_name}/{p.name}")
    return relpaths

train_rel = copy_split(train_imgs, TRAIN_DIR, "train")
val_rel   = copy_split(val_imgs,   VAL_DIR,   "val")
test_rel  = copy_split(test_imgs,  TEST_DIR,  "test")

# =========================
# SAVE splits.json
# =========================
splits = {"train": train_rel, "val": val_rel, "test": test_rel}

with open(PROCESSED_DIR / "splits.json", "w") as f:
    json.dump(
        {
            "created_at": datetime.now().isoformat(timespec="seconds"),
            "source": str(INTERIM_DIR),
            "seed": SEED,
            "ratios": {"train": TRAIN_RATIO, "val": VAL_RATIO, "test": TEST_RATIO},
            "counts": {k: len(v) for k, v in splits.items()},
            "grouping": "base_name (strip _orig/_augXX)",
            "n_groups": n_groups,
            "splits": splits,
        },
        f,
        indent=2,
    )

# =========================
# INDEX.CSV
# =========================
def extract_timestamp(filename: str) -> str:
    """
    Matches: YYYY-MM-DD_HH-MM-SS-sss.png
    (Your augmented files may NOT have this; it's okay if blank.)
    """
    m = re.search(r"(\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})-(\d{3})", filename)
    if not m:
        return ""
    return f"{m.group(1)}T{m.group(2).replace('-', ':')}.{m.group(3)}"

with open(PROCESSED_DIR / "index.csv", "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=[
            "filepath",
            "split",
            "stage",
            "scan_session_id",
            "group_id",
            "filename",
            "scan_timestamp",
            "source_interim_path",
        ],
    )
    writer.writeheader()

    for split_name, imgs in [("train", train_imgs), ("val", val_imgs), ("test", test_imgs)]:
        for p in imgs:
            gid = group_id_from_filename(p.name)
            writer.writerow(
                {
                    "filepath": f"images/{split_name}/{p.name}",
                    "split": split_name,
                    "stage": "Stage1",
                    "scan_session_id": INTERIM_DIR.name,
                    "group_id": gid,
                    "filename": p.name,
                    "scan_timestamp": extract_timestamp(p.name),
                    "source_interim_path": str(p),
                }
            )

print("✅ Dataset frozen with GROUP split (aug/orig siblings stay together)")

Found 875 images
Groups total: 175
Group split -> train/val/test: 148/17/10
Image split -> train/val/test: 740/85/50
✅ Dataset frozen with GROUP split (aug/orig siblings stay together)


In [2]:
# =========================
# ADD-ON: MERGE Stage0 processed splits into Stage1 processed splits
# ✅ NO Stage0__ / Stage0_ naming
# ✅ filenames stay as: 2026-...png
# ✅ safe: avoids overwriting collisions
# ✅ updates Stage1/splits.json + Stage1/index.csv
# =========================

from pathlib import Path
import shutil
import json
import csv

# -------------------------
# REQUIRED: these must already exist in your notebook/script
# -------------------------
# IMAGES_DIR   = Path("../../data/processed/Stage1/images")
# PROCESSED_DIR = Path("../../data/processed/Stage1")
# IMAGE_EXTS   = {".png", ".jpg", ".jpeg", ".bmp", ".tiff"}
# group_id_from_filename(filename: str) -> str
# extract_timestamp(filename: str) -> str

# -------------------------
# CONFIG
# -------------------------
STAGE0_PROCESSED_DIR = Path("../../data/processed/Stage0")
STAGE0_IMAGES_DIR = STAGE0_PROCESSED_DIR / "images"

STAGE1_PROCESSED_DIR = Path("../../data/processed/Stage1")
PROCESSED_DIR = STAGE1_PROCESSED_DIR
IMAGES_DIR = STAGE1_PROCESSED_DIR / "images"

IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tiff"}


# -------------------------
# COPY (NO RENAMING)
# -------------------------
def copy_stage0_into_stage1(split_name: str):
    src_dir = STAGE0_IMAGES_DIR / split_name
    dst_dir = IMAGES_DIR / split_name
    dst_dir.mkdir(parents=True, exist_ok=True)

    if not src_dir.exists():
        print(f"Stage0 split folder not found: {src_dir}")
        return []

    copied_relpaths = []
    skipped_collision = 0

    for p in sorted(src_dir.iterdir()):
        if p.suffix.lower() not in IMAGE_EXTS:
            continue

        # ✅ keep exact filename (no Stage0 prefix)
        out_name = p.name
        dst = dst_dir / out_name

        if dst.exists():
            # collision or already copied — skip to avoid overwriting
            skipped_collision += 1
            continue

        shutil.copy2(p, dst)
        copied_relpaths.append(f"images/{split_name}/{out_name}")

    print(f"Copied {len(copied_relpaths)} Stage0 images into Stage1/{split_name}")
    if skipped_collision:
        print(f"Skipped {skipped_collision} because filename already existed in Stage1/{split_name}")
    return copied_relpaths


# Copy Stage0 -> Stage1 for each split
stage0_train_rel = copy_stage0_into_stage1("train")
stage0_val_rel   = copy_stage0_into_stage1("val")
stage0_test_rel  = copy_stage0_into_stage1("test")


# -------------------------
# OPTIONAL: Update Stage1 splits.json
# -------------------------
splits_path = PROCESSED_DIR / "splits.json"
if splits_path.exists():
    with open(splits_path, "r") as f:
        meta = json.load(f)

    meta.setdefault("splits", {}).setdefault("train", [])
    meta.setdefault("splits", {}).setdefault("val", [])
    meta.setdefault("splits", {}).setdefault("test", [])

    meta["splits"]["train"].extend(stage0_train_rel)
    meta["splits"]["val"].extend(stage0_val_rel)
    meta["splits"]["test"].extend(stage0_test_rel)

    meta["counts"] = {k: len(v) for k, v in meta["splits"].items()}
    meta.setdefault("sources", [])
    if str(STAGE0_PROCESSED_DIR) not in meta["sources"]:
        meta["sources"].append(str(STAGE0_PROCESSED_DIR))

    with open(splits_path, "w") as f:
        json.dump(meta, f, indent=2)

    print(" Updated Stage1/splits.json with Stage0 appended")
else:
    print(" Stage1/splits.json not found; skipped updating it")


# -------------------------
# OPTIONAL: Append Stage0 rows into Stage1 index.csv
# (NO Stage0 naming; mark stage as Stage01_combined)
# -------------------------
index_path = PROCESSED_DIR / "index.csv"
if index_path.exists():
    # Read existing rows so we don't duplicate
    existing = set()
    with open(index_path, "r", newline="") as f:
        reader = csv.DictReader(f)
        for row in reader:
            existing.add(row["filepath"])

    def append_index_rows(split_name: str, relpaths: list):
        with open(index_path, "a", newline="") as f:
            writer = csv.DictWriter(
                f,
                fieldnames=[
                    "filepath",
                    "split",
                    "stage",
                    "scan_session_id",
                    "group_id",
                    "filename",
                    "scan_timestamp",
                    "source_interim_path",
                ],
            )

            for rel in relpaths:
                if rel in existing:
                    continue

                filename = Path(rel).name  # ✅ still 2026-...png

                writer.writerow(
                    {
                        "filepath": rel,
                        "split": split_name,
                        # don't label as Stage0 (prevents “Stage0 naming” vibe downstream)
                        "stage": "Stage01_combined",
                        "scan_session_id": "merged_stage0_stage1",
                        "group_id": group_id_from_filename(filename),
                        "filename": filename,
                        "scan_timestamp": extract_timestamp(filename),
                        # point back to Stage0 source file (no weird replace)
                        "source_interim_path": str(STAGE0_IMAGES_DIR / split_name / filename),
                    }
                )

    append_index_rows("train", stage0_train_rel)
    append_index_rows("val", stage0_val_rel)
    append_index_rows("test", stage0_test_rel)

    print("✅ Appended Stage0 rows into Stage1/index.csv")
else:
    print("⚠️ Stage1/index.csv not found; skipped appending")

Copied 323 Stage0 images into Stage1/train
Copied 35 Stage0 images into Stage1/val
Copied 17 Stage0 images into Stage1/test
 Updated Stage1/splits.json with Stage0 appended
✅ Appended Stage0 rows into Stage1/index.csv


Automate Labelling for Stage1 since all data will have same labels

For Train.json

In [3]:
from pathlib import Path
import json

IMG_DIR = Path("../../data/processed/Stage1/images/train")
OUT_JSON = Path("../../data/labels/Stage1/train.json")

OUT_JSON.parent.mkdir(parents=True, exist_ok=True)

annotations = []

for p in sorted(IMG_DIR.iterdir()):
    if p.suffix.lower() in {".png", ".jpg", ".jpeg"}:
        annotations.append({
            "image": p.name,
            "no_contraband": 1,
            "isolated_items": 1,
            "empty": 0
        })

with open(OUT_JSON, "w") as f:
    json.dump(annotations, f, indent=2)

print(f"✅ JSON generated with {len(annotations)} samples → {OUT_JSON}")


✅ JSON generated with 1063 samples → ../../data/labels/Stage1/train.json


For Val.json

In [4]:
from pathlib import Path
import json

IMG_DIR = Path("../../data/processed/Stage1/images/val")
OUT_JSON = Path("../../data/labels/Stage1/val.json")

OUT_JSON.parent.mkdir(parents=True, exist_ok=True)

annotations = []

for p in sorted(IMG_DIR.iterdir()):
    if p.suffix.lower() in {".png", ".jpg", ".jpeg"}:
        annotations.append({
            "image": p.name,
            "no_contraband": 1,
            "isolated_items": 1,
            "empty": 0
        })

with open(OUT_JSON, "w") as f:
    json.dump(annotations, f, indent=2)

print(f"✅ JSON generated with {len(annotations)} samples → {OUT_JSON}")


✅ JSON generated with 120 samples → ../../data/labels/Stage1/val.json


In [5]:
from pathlib import Path
import json

STAGE0_LABEL_DIR = Path("../../data/labels/Stage0")
STAGE1_LABEL_DIR = Path("../../data/labels/Stage1")

OUT_DIR = Path("../../data/labels/Stage1/combined")
SPLITS = ["train", "val"]  # add "test" if you want


def load_json(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Missing file: {path.resolve()}")
    with open(path, "r") as f:
        return json.load(f)


def save_json(path: Path, data):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        json.dump(data, f, indent=2)
    print(f"📝 Wrote: {path.resolve()} | items={len(data)}")


for split in SPLITS:
    print("\n==============================")
    print("Split:", split)

    s0_path = STAGE0_LABEL_DIR / f"{split}.json"
    s1_path = STAGE1_LABEL_DIR / f"{split}.json"
    out_path = OUT_DIR / f"{split}.json"

    s0 = load_json(s0_path)
    s1 = load_json(s1_path)

    print("Stage0:", len(s0), "|", s0_path.resolve())
    print("Stage1:", len(s1), "|", s1_path.resolve())

    # ✅ just merge lists, keep everything exactly as-is
    merged = list(s1) + list(s0)

    # optional: quick sanity counts
    print("Merged:", len(merged), "(should be Stage1 + Stage0)")

    save_json(out_path, merged)


Split: train
Stage0: 323 | /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage0/train.json
Stage1: 1063 | /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/train.json
Merged: 1386 (should be Stage1 + Stage0)
📝 Wrote: /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/combined/train.json | items=1386

Split: val
Stage0: 35 | /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage0/val.json
Stage1: 120 | /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/val.json
Merged: 155 (should be Stage1 + Stage0)
📝 Wrote: /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/combined/val.json | items=155


In [6]:
from pathlib import Path
import json

p = Path("../../data/labels/Stage1/train.json")
print("Reading:", p.resolve())
with open(p, "r") as f:
    data = json.load(f)

print("Count:", len(data))
print("First 3:", data[:3])
print("Any Stage0__ entries?", any(x["image"].startswith("Stage0__") for x in data))

Reading: /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/train.json
Count: 1063
First 3: [{'image': '2026-01-21_10-32-05-130_aug00.png', 'no_contraband': 1, 'isolated_items': 1, 'empty': 0}, {'image': '2026-01-21_10-32-05-130_aug000.png', 'no_contraband': 1, 'isolated_items': 1, 'empty': 0}, {'image': '2026-01-21_10-32-05-130_aug001.png', 'no_contraband': 1, 'isolated_items': 1, 'empty': 0}]
Any Stage0__ entries? False


In [1]:
from pathlib import Path
import shutil
import json
import csv
import random
import re
from datetime import datetime
from collections import defaultdict

# =========================
# CONFIG
# =========================
# Stage1 interim source (raw augmented outputs)
STAGE1_INTERIM_DIR = Path("../../data/interim/Stage1/color_clahe_1500x1000_noborder_aug")

# Stage1 processed output (the combined dataset will live here)
STAGE1_PROCESSED_DIR = Path("../../data/processed/Stage1")
STAGE1_IMAGES_DIR = STAGE1_PROCESSED_DIR / "images"
TRAIN_DIR = STAGE1_IMAGES_DIR / "train"
VAL_DIR   = STAGE1_IMAGES_DIR / "val"
TEST_DIR  = STAGE1_IMAGES_DIR / "test"

# Stage0 processed source to merge in (already split)
STAGE0_PROCESSED_DIR = Path("../../data/processed/Stage0")
STAGE0_IMAGES_DIR = STAGE0_PROCESSED_DIR / "images"

# Labels dirs
STAGE0_LABEL_DIR = Path("../../data/labels/Stage0")
STAGE1_LABEL_DIR = Path("../../data/labels/Stage1")
COMBINED_LABEL_DIR = Path("../../data/labels/Stage1/combined")

# Split ratios for Stage1 (group-safe)
TRAIN_RATIO = 0.85
VAL_RATIO   = 0.10
TEST_RATIO  = 0.05
SEED = 42

IMAGE_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tiff"}

# Stage1 auto-labels (your Stage1 is isolated benign items)
STAGE1_AUTO_LABEL = {
    "no_contraband": 1,
    "isolated_items": 1,
    "empty": 0,
}

# =========================
# HELPERS
# =========================
def ensure_empty_dir(d: Path):
    d.mkdir(parents=True, exist_ok=True)
    # clear folder
    for p in d.iterdir():
        if p.is_file():
            p.unlink()
        elif p.is_dir():
            shutil.rmtree(p)

def group_id_from_filename(filename: str) -> str:
    """
    Groups siblings together:
      base_orig.png, base_aug00.png, base_aug01.png -> same group "base"
    """
    stem = Path(filename).stem
    stem = re.sub(r"_(orig|aug\d{2})$", "", stem)
    return stem

def extract_timestamp(filename: str) -> str:
    """
    Matches: YYYY-MM-DD_HH-MM-SS-sss
    returns ISO-like string, else "".
    """
    m = re.search(r"(\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})-(\d{3})", filename)
    if not m:
        return ""
    return f"{m.group(1)}T{m.group(2).replace('-', ':')}.{m.group(3)}"

def load_json_list(path: Path):
    if not path.exists():
        return []
    with open(path, "r") as f:
        data = json.load(f)
    if not isinstance(data, list):
        raise ValueError(f"Expected list JSON at {path}, got {type(data)}")
    return data

def save_json_list(path: Path, data_list):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w") as f:
        json.dump(data_list, f, indent=2)

def copy_images_into_split(src_paths, dst_dir: Path, split_name: str):
    """
    Copy images into Stage1 processed split folder WITHOUT renaming.
    Skips if filename already exists in destination.
    Returns list of relpaths (images/{split}/{filename})
    """
    dst_dir.mkdir(parents=True, exist_ok=True)
    relpaths = []
    skipped = 0
    for p in sorted(src_paths, key=lambda x: x.name):
        if p.suffix.lower() not in IMAGE_EXTS:
            continue
        dst = dst_dir / p.name
        if dst.exists():
            skipped += 1
            continue
        shutil.copy2(p, dst)
        relpaths.append(f"images/{split_name}/{p.name}")
    return relpaths, skipped

def write_index_csv(index_path: Path, rows: list):
    """
    Overwrite index.csv with provided rows.
    """
    index_path.parent.mkdir(parents=True, exist_ok=True)
    with open(index_path, "w", newline="") as f:
        writer = csv.DictWriter(
            f,
            fieldnames=[
                "filepath",
                "split",
                "stage",
                "scan_session_id",
                "group_id",
                "filename",
                "scan_timestamp",
                "source_interim_path",
            ],
        )
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

# =========================
# 0) SETUP OUTPUT FOLDERS (clear Stage1 processed)
# =========================
random.seed(SEED)

ensure_empty_dir(TRAIN_DIR)
ensure_empty_dir(VAL_DIR)
ensure_empty_dir(TEST_DIR)

print("Cleared Stage1 processed split folders:", TRAIN_DIR, VAL_DIR, TEST_DIR)

# =========================
# 1) SPLIT Stage1 INTERIM INTO 3 FOLDERS (group-safe)
# =========================
stage1_image_paths = sorted([p for p in STAGE1_INTERIM_DIR.iterdir() if p.suffix.lower() in IMAGE_EXTS])
if not stage1_image_paths:
    raise ValueError(f"No images found in {STAGE1_INTERIM_DIR.resolve()}")
print(f"Stage1 interim images found: {len(stage1_image_paths)}")

groups = defaultdict(list)
for p in stage1_image_paths:
    gid = group_id_from_filename(p.name)
    groups[gid].append(p)

group_ids = list(groups.keys())
random.shuffle(group_ids)

n_groups = len(group_ids)
n_train_g = int(n_groups * TRAIN_RATIO)
n_val_g   = int(n_groups * VAL_RATIO)

train_gids = set(group_ids[:n_train_g])
val_gids   = set(group_ids[n_train_g:n_train_g + n_val_g])
test_gids  = set(group_ids[n_train_g + n_val_g:])

stage1_train_imgs = [p for gid in train_gids for p in groups[gid]]
stage1_val_imgs   = [p for gid in val_gids   for p in groups[gid]]
stage1_test_imgs  = [p for gid in test_gids  for p in groups[gid]]

print(f"Stage1 groups total: {n_groups}")
print(f"Stage1 group split -> train/val/test: {len(train_gids)}/{len(val_gids)}/{len(test_gids)}")
print(f"Stage1 image split -> train/val/test: {len(stage1_train_imgs)}/{len(stage1_val_imgs)}/{len(stage1_test_imgs)}")

stage1_train_rel, s1_skip_tr = copy_images_into_split(stage1_train_imgs, TRAIN_DIR, "train")
stage1_val_rel,   s1_skip_va = copy_images_into_split(stage1_val_imgs,   VAL_DIR,   "val")
stage1_test_rel,  s1_skip_te = copy_images_into_split(stage1_test_imgs,  TEST_DIR,  "test")

print("Copied Stage1 images into processed splits.")
if (s1_skip_tr + s1_skip_va + s1_skip_te) > 0:
    print("Skipped Stage1 collisions (unexpected since folder was cleared):",
          s1_skip_tr, s1_skip_va, s1_skip_te)

# =========================
# 2) COPY Stage0 PROCESSED INTO SAME 3 FOLDERS (NO RENAMING)
# =========================
def stage0_split_paths(split_name: str):
    d = STAGE0_IMAGES_DIR / split_name
    if not d.exists():
        print(f"Stage0 split folder missing: {d.resolve()}")
        return []
    return sorted([p for p in d.iterdir() if p.suffix.lower() in IMAGE_EXTS])

s0_train_paths = stage0_split_paths("train")
s0_val_paths   = stage0_split_paths("val")
s0_test_paths  = stage0_split_paths("test")

stage0_train_rel, s0_skip_tr = copy_images_into_split(s0_train_paths, TRAIN_DIR, "train")
stage0_val_rel,   s0_skip_va = copy_images_into_split(s0_val_paths,   VAL_DIR,   "val")
stage0_test_rel,  s0_skip_te = copy_images_into_split(s0_test_paths,  TEST_DIR,  "test")

print("Copied Stage0 images into Stage1 processed splits (no renaming).")
if (s0_skip_tr + s0_skip_va + s0_skip_te) > 0:
    print("Skipped Stage0 collisions (means same filename already existed in Stage1):",
          s0_skip_tr, s0_skip_va, s0_skip_te)

# =========================
# 3) WRITE Stage1 splits.json (combined)
# =========================
combined_splits = {
    "train": stage1_train_rel + stage0_train_rel,
    "val":   stage1_val_rel   + stage0_val_rel,
    "test":  stage1_test_rel  + stage0_test_rel,
}

splits_json = {
    "created_at": datetime.now().isoformat(timespec="seconds"),
    "seed": SEED,
    "ratios_stage1": {"train": TRAIN_RATIO, "val": VAL_RATIO, "test": TEST_RATIO},
    "counts": {k: len(v) for k, v in combined_splits.items()},
    "grouping_stage1": "base_name (strip _orig/_augXX)",
    "n_groups_stage1": n_groups,
    "sources": [str(STAGE1_INTERIM_DIR), str(STAGE0_PROCESSED_DIR)],
    "splits": combined_splits,
}

with open(STAGE1_PROCESSED_DIR / "splits.json", "w") as f:
    json.dump(splits_json, f, indent=2)

print("Wrote:", (STAGE1_PROCESSED_DIR / "splits.json").resolve())

# =========================
# 4) WRITE Stage1 index.csv (combined)
#    - includes Stage1 + Stage0 copied items
# =========================
index_rows = []

def add_index_rows(relpaths, split_name, stage_name, scan_session_id, source_root: Path):
    for rel in relpaths:
        filename = Path(rel).name
        index_rows.append(
            {
                "filepath": rel,
                "split": split_name,
                "stage": stage_name,
                "scan_session_id": scan_session_id,
                "group_id": group_id_from_filename(filename),
                "filename": filename,
                "scan_timestamp": extract_timestamp(filename),
                "source_interim_path": str(source_root / split_name / filename) if (source_root / split_name).exists() else "",
            }
        )

# Stage1 rows (source is the interim dir itself, but we only have filename; keep absolute original if possible)
# We'll set source_interim_path as STAGE1_INTERIM_DIR/<filename> (best effort).
def add_stage1_index(relpaths, split_name):
    for rel in relpaths:
        filename = Path(rel).name
        index_rows.append(
            {
                "filepath": rel,
                "split": split_name,
                "stage": "Stage1",
                "scan_session_id": STAGE1_INTERIM_DIR.name,
                "group_id": group_id_from_filename(filename),
                "filename": filename,
                "scan_timestamp": extract_timestamp(filename),
                "source_interim_path": str(STAGE1_INTERIM_DIR / filename),
            }
        )

add_stage1_index(stage1_train_rel, "train")
add_stage1_index(stage1_val_rel, "val")
add_stage1_index(stage1_test_rel, "test")

# Stage0 rows
add_index_rows(stage0_train_rel, "train", "Stage0", STAGE0_PROCESSED_DIR.name, STAGE0_IMAGES_DIR)
add_index_rows(stage0_val_rel,   "val",   "Stage0", STAGE0_PROCESSED_DIR.name, STAGE0_IMAGES_DIR)
add_index_rows(stage0_test_rel,  "test",  "Stage0", STAGE0_PROCESSED_DIR.name, STAGE0_IMAGES_DIR)

index_path = STAGE1_PROCESSED_DIR / "index.csv"
write_index_csv(index_path, index_rows)
print("Wrote:", index_path.resolve(), "| rows:", len(index_rows))

# =========================
# 5) AUTO-GENERATE Stage1 labels (train/val/test) based on Stage1 PROCESSED folders
#    (All same labels you specified)
# =========================
def write_auto_label_json(img_dir: Path, out_json: Path):
    anns = []
    for p in sorted(img_dir.iterdir()):
        if p.suffix.lower() in IMAGE_EXTS:
            anns.append({"image": p.name, **STAGE1_AUTO_LABEL})
    save_json_list(out_json, anns)
    print(f"Auto labels: {out_json.resolve()} | items={len(anns)}")
    return anns

stage1_train_json = STAGE1_LABEL_DIR / "train.json"
stage1_val_json   = STAGE1_LABEL_DIR / "val.json"
#stage1_test_json  = STAGE1_LABEL_DIR / "test.json"

_ = write_auto_label_json(TRAIN_DIR, stage1_train_json)
_ = write_auto_label_json(VAL_DIR,   stage1_val_json)
#_ = write_auto_label_json(TEST_DIR,  stage1_test_json)

# =========================
# 6) MERGE Stage1 + Stage0 label JSONs into combined/
#    (keeps entries exactly as-is; no renaming; duplicates allowed)
# =========================
COMBINED_LABEL_DIR.mkdir(parents=True, exist_ok=True)

def merge_label_lists(stage1_path: Path, stage0_path: Path, out_path: Path):
    s1 = load_json_list(stage1_path)
    s0 = load_json_list(stage0_path)
    merged = list(s1) + list(s0)  # EXACTLY as-is
    save_json_list(out_path, merged)
    print(f"Merged labels -> {out_path.resolve()} | Stage1={len(s1)} + Stage0={len(s0)} = {len(merged)}")

for split in ["train", "val"]:
    merge_label_lists(
        STAGE1_LABEL_DIR / f"{split}.json",
        STAGE0_LABEL_DIR / f"{split}.json",
        COMBINED_LABEL_DIR / f"{split}.json",
    )

print("\nDONE: Stage1 processed now contains Stage1+Stage0 images in 3 folders, and splits/index/labels are updated.")

Cleared Stage1 processed split folders: ../../data/processed/Stage1/images/train ../../data/processed/Stage1/images/val ../../data/processed/Stage1/images/test
Stage1 interim images found: 875
Stage1 groups total: 175
Stage1 group split -> train/val/test: 148/17/10
Stage1 image split -> train/val/test: 740/85/50
Copied Stage1 images into processed splits.
Copied Stage0 images into Stage1 processed splits (no renaming).
Wrote: /home/ssy/Desktop/xray-gen-ai_Project/data/processed/Stage1/splits.json
Wrote: /home/ssy/Desktop/xray-gen-ai_Project/data/processed/Stage1/index.csv | rows: 2056
Auto labels: /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/train.json | items=1715
Auto labels: /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/val.json | items=221
Merged labels -> /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/combined/train.json | Stage1=1715 + Stage0=975 = 2690
Merged labels -> /home/ssy/Desktop/xray-gen-ai_Project/data/labels/Stage1/combined/val.json | S